### GOLD Layer: Aggregated & Analytical Views

In [19]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = SparkSession.builder.appName("Bronze Ingestion") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("WARN")
spark.conf.set("spark.sql.shuffle.partitions", "4")

### Load SILVER Delta Tables

In [20]:
base_df = spark.read.format("delta") \
    .load("../delta/02_silver/orders_enriched")

In [21]:
# Derive partition columns
base_df = (
    base_df
    .withColumn("year", F.year("order_purchase_timestamp"))
    .withColumn("month", F.month("order_purchase_timestamp"))
    .withColumn("day", F.dayofmonth("order_purchase_timestamp"))
)

### 1. Cumulative Sales per Customer

In [22]:
from pyspark.sql import Window

customer_window = (
    Window
    .partitionBy("customer_id")
    .orderBy("order_purchase_timestamp")
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

cumulative_sales_df = (
    base_df
    .select(
        "customer_id",
        "order_id",
        "order_purchase_timestamp",
        "total_price",
        "year", "month", "day",
        F.sum("total_price")
         .over(customer_window)
         .alias("cumulative_sales")
    )
)

cumulative_sales_df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month", "day") \
    .save("../delta/03_gold/cumulative_sales_per_customer")

### 2. Rolling Average Delivery Time per Product Category (last 3 entries)

In [23]:
category_window = (
    Window
    .partitionBy("product_category_name")
    .orderBy("order_purchase_timestamp")
    .rowsBetween(-2, 0)
)

rolling_delivery_df = (
    base_df
    .select(
        "product_category_name",
        "order_id",
        "order_purchase_timestamp",
        "delivery_time_days",
        "year", "month", "day",
        F.avg("delivery_time_days")
         .over(category_window)
         .alias("rolling_avg_delivery_time")
    )
)

rolling_delivery_df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month", "day") \
    .save("../delta/03_gold/rolling_avg_delivery_time_per_category")

### 3. KPI Summary Tables


### 3.1 Total sales per product category

In [24]:
sales_per_category_df = (
    base_df
    .groupBy("product_category_name", "year", "month", "day")
    .agg(F.sum("total_price").alias("total_sales"))
)

sales_per_category_df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month", "day") \
    .save("../delta/03_gold/total_sales_per_category")


### 3.2 Average delivery time per seller

In [25]:
avg_delivery_per_seller_df = (
    base_df
    .groupBy("seller_id", "year", "month", "day")
    .agg(F.avg("delivery_time_days").alias("avg_delivery_time"))
)

avg_delivery_per_seller_df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month", "day") \
    .save("../delta/03_gold/avg_delivery_time_per_seller")


### 3.3 Order counts per customer state

In [26]:
orders_per_state_df = (
    base_df
    .groupBy("customer_state", "year", "month", "day")
    .agg(F.countDistinct("order_id").alias("order_count"))
)

orders_per_state_df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month", "day") \
    .save("../delta/03_gold/order_counts_per_customer_state")

### Register GOLD tables for SQL queries


In [28]:
spark.read.format("delta") \
    .load("../delta/03_gold/total_sales_per_category") \
    .createOrReplaceTempView("gold_total_sales_per_category")

spark.read.format("delta") \
    .load("../delta/03_gold/avg_delivery_time_per_seller") \
    .createOrReplaceTempView("gold_avg_delivery_time_per_seller")

spark.read.format("delta") \
    .load("../delta/03_gold/order_counts_per_customer_state") \
    .createOrReplaceTempView("gold_order_counts_per_customer_state")

### Reporting Queries (SQL)

In [29]:

# Total sales per product category
total_sales_per_category_sql = spark.sql("""
SELECT
    product_category_name,
    SUM(total_sales) AS total_sales
FROM gold_total_sales_per_category
GROUP BY product_category_name
ORDER BY total_sales DESC
""")

total_sales_per_category_sql.show(truncate=False)


# Average delivery time per seller
avg_delivery_time_per_seller_sql = spark.sql("""
SELECT
    seller_id,
    AVG(avg_delivery_time) AS avg_delivery_time_days
FROM gold_avg_delivery_time_per_seller
GROUP BY seller_id
ORDER BY avg_delivery_time_days
""")

avg_delivery_time_per_seller_sql.show(truncate=False)


# Number of orders by customer state
orders_by_state_sql = spark.sql("""
SELECT
    customer_state,
    SUM(order_count) AS total_orders
FROM gold_order_counts_per_customer_state
GROUP BY customer_state
ORDER BY total_orders DESC
""")

orders_by_state_sql.show(truncate=False)

+----------------------+------------------+
|product_category_name |total_sales       |
+----------------------+------------------+
|beleza_saude          |1441248.07        |
|relogios_presentes    |1305541.61        |
|cama_mesa_banho       |1241681.7200000002|
|esporte_lazer         |1156656.4800000002|
|informatica_acessorios|1059272.4         |
|moveis_decoracao      |902511.7900000003 |
|utilidades_domesticas |778397.7699999999 |
|cool_stuff            |719329.9500000001 |
|automotivo            |685384.3200000001 |
|ferramentas_jardim    |584219.2099999998 |
|brinquedos            |561372.5499999999 |
|bebes                 |480118.0          |
|perfumaria            |453338.7099999999 |
|telefonia             |394883.32         |
|moveis_escritorio     |342532.65         |
|papelaria             |277741.71         |
|pet_shop              |253876.65         |
|pcs                   |232799.42999999996|
|instrumentos_musicais |210137.37000000002|
|NULL                  |207705.0

In [30]:
orders_by_state_sql.toPandas()

,customer_state,total_orders
0,SP,41375
1,RJ,12762
2,MG,11544
3,RS,5432
4,PR,4998
5,SC,3612
6,BA,3358
7,DF,2125
8,ES,2025
9,GO,2007
